In [1]:
#ライブラリーとデータの読み込み
import pandas as pd

transactions = pd.read_csv("../input/rakusp/for_uploading/transactions_rakus_train.csv")
articles = pd.read_csv("../input/rakusp/for_uploading/articles.csv")
customers_rakus = pd.read_csv("../input/rakusp/for_uploading/customers_rakus.csv")

In [2]:
transactions

In [3]:
customers_rakus

1. 購買履歴が直近3ヶ月間で存在するユーザー(active)

In [ ]:
# 日付（'t_dat'列）をdatetime型に変換
import datetime
transactions['t_dat'] = pd.to_datetime(transactions['t_dat'], format='%Y-%m-%d')

In [ ]:
# 直近3か月間の購買履歴が存在するユーザーの絞り込み
date_3m_before = datetime.datetime(2020, 6, 7)
customers_within_3M = transactions[transactions["t_dat"] >= date_3m_before].reset_index(drop = True).customer_id
customers_recent = customers_within_3M.value_counts()
customers_recent

2. 購買履歴が直近3ヶ月より前には存在するが、3ヶ月間では無いユーザー(inactive)

In [ ]:
# 直近3か月より前の購買履歴に絞り込み
transactions_before_3M_ago = transactions[transactions['t_dat'] < date_3m_before]
transactions_old = transactions_before_3M_ago[~transactions_before_3M_ago['customer_id'].isin(customers_within_3M)]
customers_before_3M_ago = transactions_old['customer_id']
customers_old = customers_before_3M_ago.value_counts()
customers_old

3. 購入履歴無しの顧客

In [ ]:
#購買履歴のある顧客IDを抽出
action_customer_ID = transactions["customer_id"].unique()

#購買履歴のない顧客IDを抽出
non_action_customers_rakus_ID = customers_rakus[~customers_rakus["customer_id"].isin(action_customer_ID)].reset_index(drop=True)

In [ ]:
#同じ商品を10個以上購入している顧客の取り出し
unique_users = transactions.groupby(["customer_id","article_id"],as_index=False).count().drop(["price","sales_channel_id"],axis=1)
unique_users.columns=["customer_id","article_id","count"]
unique_users = unique_users.sort_values(by='count',ascending=False)

#同じ商品を10個以上購入している顧客IDの抽出
irregular_action_customers_rakus_ID = customers_rakus[customers_rakus["customer_id"].isin(unique_users["customer_id"][unique_users["count"]>=10])].reset_index(drop=True)